### 모델 매개변수 최적화하기(OPTIMIZING MODEL PARAMETERS)
* 데이터에 매개변수를 최적화하여 모델을 학습, 검증, 테스트 
* 모델을 학습하는 과정은 반복적인 과정을 거친다.
 - 각 반복단계인 epoch에서 모델은 출력을 추측하고(guess about the output), 
 - 추측과 정답사이의 오류(calculates the error in its guess)인 loss를 계산
 - 매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤,
 - 경사하강법을 사용하여 이 parameter들을 optimize함
 

In [ ]:
from numpy import False_
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits 

model=NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### Hyperparameter
학습 시 다음과 같은 hyperparameter정의
- epoch 수 : 데이터셋을 반복하는 횟수
- batch size : 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 개수
- learning rate: 각 batch/epoch에서 모델의 매개변수를 조절하는 비율. 
 - 작을수록 학습속도 느려지고, 커지면 예측할 수 없는 동작 발생할 수 있음

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 최적화 단계 (Optimization Loop)
* hyperparameter설정 뒤에 최적화 단계를 통해 모델을 학습하고 최적화할 수 있음.
* 최적화 단계의 각 반복(iteration)을 epoch이라고 함 

* 하나의 에폭은 두 부분으로 구성됨
 * 학습단계(train loop) - training dataset을 반복(iterate)하고 최적의 매개변수로 수렴함
 * validation/test loop : 모델 성능이 개선되고 있는지를 확인하기 위해 test dataset을 iterate함

#### 손실함수(loss function)
* 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산함.



In [ ]:
loss_fn = nn.CrossEntropyLoss()

#### Optimizer
* 각 학습단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
* 최적화 알고리즘 - 확률적 경사하강법(SGD) 
* ADAM, RMSProp .. 다른종류의 모델과 데이터에서 더 잘 작동하는 다양한 옵티마이저들이 존재

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

* 학습단계(loop)에서 최적화는 세단계로 이루어짐
 - optimizer.zero_grad() 를 호출하여 모델 매개변수의 변화도를 재설정함.
 - loss.backwards() 를 호출하여 예측손실(prediction loss)를 역전파함.
 - 변화도를 계산한 후 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정함.
 

#### 전체 구현

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset) #number of records
  for batch, (X, y) in enumerate(dataloader):
    #prediction과 loss 계산
    pred = model(X)  #model = NeuralNetwork().to(device)
    loss = loss_fn(pred, y)

    #역전파
    optimizer.zero_grad() #grad 축적 방지(iteration마다 초기화)
    loss.backward()   # loss를 이용해 backprop을 진행함으로써 최적의 parameter값들을 찾음
    optimizer.step()  # 최적의 값들로 parameter들을 update

    if batch % 100 == 0 :  # 100 iteration마다 다음의 값들을 출력해라
      loss, current = loss.item(), batch * len(X)
      #current: batch*len(X) - 현재까지 학습한 data개수
      #len(X): batch_size * 100

      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn): #train_loop에서 optimizer만 빠짐
    size = len(dataloader.dataset)
    num_batches = len(dataloader)  # number_of_data/ batch_size
    test_loss, correct = 0, 0

    with torch.no_grad():  #validation/test data이므로 backprop이 필요없음 - grad계산 필요없음
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item() #1 epoch에 대한 test loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() 
            '''(pred.argmax(1) == y) : batch size(64)만큼의 길이를 갖는 tensor,
            각 batch의 자리에 둘이 일치하면 1, 불일치하면 0을 넣어줌

            type(torch.float).sum().item() : 텐서의 datatype을 float로 바꾼 뒤, tensor내 값을 모두 더해준 것

            correct += :  (+=) ->  1 epoch에서의 pred=True인 개수
            '''

    test_loss /= num_batches # 1epoch에 대한 average test loss를 구함
    correct /= size   # 1epoch에 대한 accuracy 구함
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epochs {t+1}\n ----------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("DONE")

Epochs 1
 ----------------------
loss: 2.177086  [    0/60000]
loss: 2.160049  [ 6400/60000]
loss: 2.109275  [12800/60000]
loss: 2.128019  [19200/60000]
loss: 2.077543  [25600/60000]
loss: 2.028062  [32000/60000]
loss: 2.054775  [38400/60000]
loss: 1.986684  [44800/60000]
loss: 1.993935  [51200/60000]
loss: 1.918303  [57600/60000]
Test Error: 
 Accuracy: 57.3%, Avg loss: 1.917031 

Epochs 2
 ----------------------
loss: 1.950506  [    0/60000]
loss: 1.911782  [ 6400/60000]
loss: 1.804818  [12800/60000]
loss: 1.846849  [19200/60000]
loss: 1.736768  [25600/60000]
loss: 1.690761  [32000/60000]
loss: 1.707062  [38400/60000]
loss: 1.618568  [44800/60000]
loss: 1.640025  [51200/60000]
loss: 1.529380  [57600/60000]
Test Error: 
 Accuracy: 61.8%, Avg loss: 1.547892 

Epochs 3
 ----------------------
loss: 1.613510  [    0/60000]
loss: 1.567762  [ 6400/60000]
loss: 1.426832  [12800/60000]
loss: 1.500220  [19200/60000]
loss: 1.379549  [25600/60000]
loss: 1.369071  [32000/60000]
loss: 1.377150  [

----

### 모델 저장하고 불러오기

In [ ]:
import torch
import torchvision.models as models

모델 가중치 저장하고 불러오기   
- 학습된 매개변수를 state_dict라고 불리는 internal state dictionary에 저장함.
- torch.save 메소드를 사용하여 저장할 수 있음.

In [ ]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

- 모델 가중치를 불러오기 위해서는, 먼저 동일한 모델의 인스턴스(instance)를 생성한 다음에 load_state_dict() 메소드를 사용하여 매개변수를 불러옴.

In [ ]:
model = models.vgg16()  #기본 가중치를 불러오지 않으므로 pretrained=True를 지정하지 않음
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

- 모델을 형태를 포함하여 저장하고 불러오기
 - 모델의 가중치를 불러올 때 신경망의 구조를 정의하기 위해 모델 클래스를 먼저 생성(instantiate)해야 했는데, 이 클래스의 구조를 모델과 함께 저장하고 싶으면, (model.state_dict()가 아닌) model 을 저장 함수에 전달한다.

In [ ]:
torch.save(model, 'model_pth')
# or
# model = torch.load('model.pth)
